# Tensorizing Flow Numerical Experiments Result

In [ ]:
import numpy as np
import scipy
import scipy.io
# import utility functions
import utils
from utils.flow_models import *
from utils.target import *
from utils.training import *

# import pytorch
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import torch.distributions as distrib
import torch.distributions.transforms as transform
# set number of threads
torch.set_num_threads(8)

# import plotting
import matplotlib.pyplot as plt
%matplotlib inline

# import I/O
import os
import sys

# helper function for unpacking reports
def load_report(report_filename, verbose=True, debug=False):
    """ Assumes:
    'initial_loss', 
    'model_snapshots', 
    'training_loss', 
    'test_loss', 
    'post_training_samples', 
    'grad_norms' variables are contained. 
    
    If debug is set to True, loads the simplified dataset.
    
    """
    report = scipy.io.loadmat(report_filename)
    plt.figure(1, figsize=(12, 6))
    train = report['training_loss'][0]
    test = report['test_loss'][0]
    if debug:
        # get rid of zero values 
        train = train[train != 0]
        test = test[test != 0]
    num_epochs = len(train)
    if not debug:
        # initial loss
        loss_initial = report['initial_loss'][0]
        print("[#####] ( Initial Loss ) = {}".format(loss_initial))
    
    plt.plot(range(num_epochs), train, label='Training')
    if not debug:
        plt.axhline(y=loss_initial, linestyle='-.', color='purple', label='Initial Loss')
    plt.plot(range(num_epochs), test, label='Generalization')
    plt.title("Loss Profile for {} Epochs".format(num_epochs))
    plt.xlabel('Epoch #'); plt.ylabel("Loss ( KL Divergence )");
    plt.legend(bbox_to_anchor=(1.1, 1.05))
    plt.grid(True)
    plt.show()
    
    # plot gradient behavior
    plt.figure(2, figsize=(12, 6))
    # get gradient information
    grad_norms = report['grad_norms'][0]
    if debug:
        # get rid of zero values 
        grad_norms = grad_norms[grad_norms != 0]
    plt.plot(range(num_epochs), grad_norms, color='blue')
    plt.xlabel('Epoch #'); plt.ylabel("Norm of Model Gradient");
    plt.title("Gradient Profile for {} Epochs".format(num_epochs))
    plt.grid(True)
    plt.show()
    
    if verbose:
        # report final profiles
        print("[#####] Final Loss Values: ( Train ) = {}, ( Test ) = {}".format(train[-1], test[-1]))
    
def compare(report_filename1, report_filename2, true_stats=None):
    """ Plot comparison plots for tensorizing and normalizing flow. 
    Assumes report1 is tensorizing flow, report 2 is normalizing flow.
    """
    # static data must exist
    assert os.path.isfile("./utils/full_rank_stats.mat"), \
    "::True statistics must be computed before loading report. "
    
    report1 = scipy.io.loadmat(report_filename1)
    report2 = scipy.io.loadmat(report_filename2)
    # compare loss profiles
    train1 = report1['training_loss'][0]
    test1 = report1['test_loss'][0]
    train2 = report2['training_loss'][0]
    test2 = report2['test_loss'][0]
    num_epochs = len(train1)
    loss_initial1 = report1['initial_loss'][0]
    loss_initial2 = report2['initial_loss'][0]
    print("=== Epoch 1 Tensorizing: Train = {}, Test = {}".format(train1[0], test1[0]))
    print("=== Epoch 1 Normalizing: Train = {}, Test = {}".format(train2[0], test2[0]))
    # subplots
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 6));
    
    
    ax1.axhline(y=loss_initial1, linestyle='-.', color='purple', label='Initial Loss (Tensor)')
    ax1.plot(range(num_epochs), test1, label='Generalization (Tensor)', color='blue')
    
    ax1.axhline(y=loss_initial2, linestyle='-.', color='black', label='Initial Loss (Normal)')
    ax1.plot(range(num_epochs), test2, label='Generalization (Normal)', color='grey')
    
    ax1.set_title("Generalization Loss Profile Comparison for {} Epochs".format(num_epochs))
    ax1.set_xlabel('Epoch #'); ax1.set_ylabel("Loss ( KL Divergence )");
    ax1.legend()
    ax1.grid(True);
    
    # zoomed in
    ax2.plot(range(num_epochs)[int(0.3*num_epochs):], \
             test1[int(0.3*num_epochs):], label='Generalization (Tensor)', color='blue')
    ax2.plot(range(num_epochs)[int(0.3*num_epochs):], \
             test2[int(0.3*num_epochs):], label='Generalization (Normal)', color='grey')
    # load true statistics
    if true_stats is not None:
        assert true_stats in ['gl1d', 'gl2d', 'rosen', 'double_rosen']
        # need to make sure data is generated
        stats = scipy.io.loadmat('./utils/full_rank_stats.mat')
        # optimal loss
        optimal_loss = stats[true_stats]['loss'][0][0][0][0]
        print("Optimum = {}".format(optimal_loss))
        print("::Displaying optimum loss for: {}, for initialization. \n\n".format(true_stats))
        print("::( Tensorizing ) % Reduction = {}, % Distance from Optimum = {}"
              .format( (loss_initial1 - test1[-1]) / loss_initial1, abs(optimal_loss - test1[-1]) / optimal_loss ))
        print("::( Normalizing ) % Reduction = {}, % Distance from Optimum = {}"
              .format( (loss_initial2 - test2[-1]) / loss_initial2, abs(optimal_loss - test2[-1]) / optimal_loss ))
        ax2.axhline(y=loss_initial1, linestyle='-.', color='purple', label='Initial Loss (TT)')
        ax2.axhline(y=loss_initial2, linestyle='-.', color='black', label='Initial Loss (Normal)')
        ax2.axhline(y=optimal_loss, linestyle='-.', color='green', label='Optimal Loss')
            
    
    ax2.set_title("Generalization Loss Profile Comparison for {} Epochs (Zoomed In)".format(num_epochs))
    ax2.set_xlabel('Epoch #'); ax2.set_ylabel("Loss ( KL Divergence )");
    # focus 
    if true_stats == 'gl1d':
        ax2.set_ylim(60, 90)
    if true_stats == 'gl2d':
        ax2.set_ylim(60, 100)
    if true_stats == 'rosen':
        ax2.set_ylim(120, 140)
    if true_stats == 'double_rosen':
        ax2.set_ylim(118, 140)
    ax2.legend()
    ax2.grid(True);
    
    

In [ ]:
%%javascript
// disable autoscrolling
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

# Aggregate Results (Planar)

Plotting loss profiles over 9 seeds. The following sections must be run sequentially.

## GL 1d

#### Normal

In [ ]:
avg_train = 0
avg_test = 0
for seed in np.arange(1, 10):
    test_file = "./report_aggregate/report/" + "seed{}_ginz1d_nf_planar_report.mat".format(seed)
    # load
    data = scipy.io.loadmat(test_file)
    plt.figure(1)
    plt.plot(np.arange(len(data['training_loss'][0])), data['training_loss'][0])
    plt.title("Training Error")
    plt.figure(2)
    plt.plot(np.arange(len(data['test_loss'][0])), data['test_loss'][0])
    plt.title("Generalization Error")
    # averaged curve
    avg_train = avg_train + data['training_loss'][0]
    avg_test = avg_test + data['test_loss'][0]
    
# plot averaged curve
avg_train_nf = avg_train / 10
avg_test_nf = avg_test / 10
plt.figure(3)
plt.plot(np.arange(len(data['training_loss'][0])), avg_train_nf, label="( Train )")
plt.plot(np.arange(len(data['training_loss'][0])), avg_test_nf, label="( Test )")
print("> Averaged Final Training Loss = {}, Test Loss = {}".format(avg_train_nf[-1], avg_test_nf[-1]))
plt.legend()

#### Tensor

#### Overlay Test Error

In [ ]:
seed1_ginz1d_rk3_tf_resnet_report

## GL 2d

#### Normal

In [ ]:
avg_train = 0
avg_test = 0
for seed in np.arange(1, 10):
    test_file = "./report_aggregate/report/" + "seed{}_ginz2d_nf_planar_report.mat".format(seed)
    # load
    data = scipy.io.loadmat(test_file)
    plt.figure(1)
    plt.plot(np.arange(len(data['training_loss'][0])), data['training_loss'][0])
    plt.title("Training Error")
    plt.figure(2)
    plt.plot(np.arange(len(data['test_loss'][0])), data['test_loss'][0])
    plt.title("Generalization Error")
    # averaged curve
    avg_train = avg_train + data['training_loss'][0]
    avg_test = avg_test + data['test_loss'][0]
    
# plot averaged curve
avg_train_nf = avg_train / 10
avg_test_nf = avg_test / 10
plt.figure(3)
plt.plot(np.arange(len(data['training_loss'][0])), avg_train_nf, label="( Train )")
plt.plot(np.arange(len(data['training_loss'][0])), avg_test_nf, label="( Test )")
print("> Averaged Final Training Loss = {}, Test Loss = {}".format(avg_train_nf[-1], avg_test_nf[-1]))
plt.legend()

#### Tensor

In [ ]:
avg_train = 0
avg_test = 0
for seed in np.arange(1, 10):
    test_file = "./report_aggregate/report/" + "seed{}_ginz2d_rk3_tf_planar_report.mat".format(seed)
    # load
    data = scipy.io.loadmat(test_file)
    plt.figure(1)
    plt.plot(np.arange(len(data['training_loss'][0])), data['training_loss'][0])
    plt.title("Training Error")
    plt.figure(2)
    plt.plot(np.arange(len(data['test_loss'][0])), data['test_loss'][0])
    plt.title("Generalization Error")
    # averaged curve
    avg_train = avg_train + data['training_loss'][0]
    avg_test = avg_test + data['test_loss'][0]
    
# plot averaged curve
avg_train_tf = avg_train / 9
avg_test_tf = avg_test / 9
plt.figure(3)
plt.plot(np.arange(len(data['training_loss'][0])), avg_train_tf, label="( Train )")
plt.plot(np.arange(len(data['training_loss'][0])), avg_test_tf, label="( Test )")
print("> Averaged Final Training Loss = {}, Test Loss = {}".format(avg_train_tf[-1], avg_test_tf[-1]))
plt.legend()

#### Overlay Test Error

In [ ]:
plt.figure(5)
plt.plot(np.arange(len(data['training_loss'][0])), avg_test_tf, label="Tensor")
plt.plot(np.arange(len(data['training_loss'][0])), avg_test_nf, label="Normal")
plt.legend()

## Rosenbrock

#### Normal

In [ ]:
avg_test_normal = 0
avg_test_tensor = 0
for seed in np.arange(1, 10):
    normal_file = "./report_aggregate/report/" + "seed{}_rosen_nf_planar_report.mat".format(seed)
    tensor_file = "./report_aggregate/report/" + "seed{}_rosen_rk5_tf_planar_report.mat".format(seed)
    # load
    normal_data = scipy.io.loadmat(normal_file)
    tensor_data = scipy.io.loadmat(tensor_file)
    #plt.figure(1)
    #plt.plot(np.arange(len(data['training_loss'][0])), data['training_loss'][0])
    #plt.title("Training Error")
    plt.figure(2, figsize=(20, 8))
    normal_test_loss = np.append(normal_data['initial_loss'][0], normal_data['test_loss'][0])
    tensor_test_loss = np.append(tensor_data['initial_loss'][0], tensor_data['test_loss'][0])
    #test_loss = data['test_loss'][0]
    grid = np.arange(len(normal_test_loss))
    #error = np.random.normal(0, 2, size=test_loss.shape)
    plt.plot(grid, normal_test_loss, color='grey')
    plt.plot(grid, tensor_test_loss, color='black')
    #plt.fill_between(grid, test_loss-error, test_loss+error, color='grey')
    plt.grid(True)
    csfont = {'fontname':'Times New Roman'}
    plt.title("Rosenbrock Test Err.", **csfont)
    # averaged curve
    avg_test_tensor = avg_test_tensor + tensor_test_loss
    avg_test_normal = avg_test_normal + normal_test_loss
    
# plot averaged curve
#avg_train_nf = avg_train / 9
avg_test_nf = avg_test_normal / 9
avg_test_tf = avg_test_tensor / 9
plt.figure(2)
plt.plot(grid, avg_test_nf, label="( Normalizing Average )", color='yellow')
plt.plot(grid, avg_test_tf, label="( Tensorizing Average )", color='orange')
print("> Averaged Final Test Loss = {}".format(avg_test_nf[-1]))
print("> Averaged Final Test Loss = {}".format(avg_test_tf[-1]))
plt.xlabel("Epoch", **csfont)
plt.ylabel("Generalization Loss", **csfont)
plt.legend()

### Plot Final Samples

In [ ]:
normal_final_samples = normal_data
tensor_final_samples = tensor_data

plt.figure(3, figsize=(10, 8))
fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(20, 5))
plt.suptitle("Planar-Normal: Post-Training", **csfont)

ax[0].scatter(normal_final_samples['post_training_samples'][:, -4], \
            normal_final_samples['post_training_samples'][:, -3], s=1., color='blue', label='normal');
ax[0].scatter(tensor_final_samples['post_training_samples'][:, -4], \
            tensor_final_samples['post_training_samples'][:, -3], s=1., color='red', label='tensor');
ax[0].grid(True)
ax[0].set_title("Dimension (8, 9)")

ax[1].scatter(normal_final_samples['post_training_samples'][:, -2], \
            normal_final_samples['post_training_samples'][:, -3], s=1., color='blue');
ax[1].scatter(tensor_final_samples['post_training_samples'][:, -2], \
            tensor_final_samples['post_training_samples'][:, -3], s=1., color='red');
ax[1].grid(True)
ax[1].set_title("Dimension (9, 10)")

ax[2].scatter(normal_final_samples['post_training_samples'][:, -2], \
            normal_final_samples['post_training_samples'][:, -1], s=1., color='blue', label='normal');
ax[2].scatter(tensor_final_samples['post_training_samples'][:, -2], \
            tensor_final_samples['post_training_samples'][:, -1], s=1., color='red', label='tensor');
ax[2].legend()
ax[2].grid(True)
ax[2].set_title("Last Dimension (10, 11)")
plt.grid(True)

#### Overlay Test Error

In [ ]:
plt.figure(5)

fig, ax = plt.subplots(1, 2, figsize=(15, 4))
plt.suptitle("Loss Profiles of Tensor vs. Normal");
ax[0].plot(grid, avg_test_tf, label="Tensor")
ax[0].plot(grid, avg_test_nf, label="Normal")
ax[0].set_title("Full Profile")
ax[0].set_ylabel("Test Loss")
ax[0].set_xlabel("Epoch")
ax[0].grid(True)

ax[1].plot(grid[50:100], avg_test_tf[50:100], label="Tensor")
ax[1].plot(grid[50:100], avg_test_nf[50:100], label="Normal")
ax[1].set_title("Zoomed In")
ax[1].set_ylabel("Test Loss")
ax[1].set_xlabel("Epoch")
ax[1].grid(True)
ax[1].legend()

## Double Rosenbrock

#### Normal

In [ ]:
avg_test_normal = 0
avg_test_tensor = 0
for seed in np.arange(1, 10):
    normal_file = "./report_aggregate/report/" + "seed{}_double_rosen_nf_planar_report.mat".format(seed)
    tensor_file = "./report_aggregate/report/" + "seed{}_double_rosen_rk4_tf_planar_report.mat".format(seed)
    # load
    normal_data = scipy.io.loadmat(normal_file)
    tensor_data = scipy.io.loadmat(tensor_file)
    #plt.figure(1)
    #plt.plot(np.arange(len(data['training_loss'][0])), data['training_loss'][0])
    #plt.title("Training Error")
    plt.figure(2, figsize=(20, 8))
    normal_test_loss = np.append(normal_data['initial_loss'][0], normal_data['test_loss'][0])
    tensor_test_loss = np.append(tensor_data['initial_loss'][0], tensor_data['test_loss'][0])
    #test_loss = data['test_loss'][0]
    grid = np.arange(len(normal_test_loss))
    #error = np.random.normal(0, 2, size=test_loss.shape)
    plt.plot(grid, normal_test_loss, color='grey')
    plt.plot(grid, tensor_test_loss, color='black')
    #plt.fill_between(grid, test_loss-error, test_loss+error, color='grey')
    plt.grid(True)
    csfont = {'fontname':'Times New Roman'}
    plt.title("Rosenbrock Test Err.", **csfont)
    # averaged curve
    avg_test_tensor = avg_test_tensor + tensor_test_loss
    avg_test_normal = avg_test_normal + normal_test_loss
    
# plot averaged curve
#avg_train_nf = avg_train / 9
avg_test_nf = avg_test_normal / 9
avg_test_tf = avg_test_tensor / 9
plt.figure(2)
plt.plot(grid, avg_test_nf, label="( Normalizing Average )", color='yellow')
plt.plot(grid, avg_test_tf, label="( Tensorizing Average )", color='orange')
print("> Averaged Final Test Loss = {}".format(avg_test_nf[-1]))
print("> Averaged Final Test Loss = {}".format(avg_test_tf[-1]))
plt.xlabel("Epoch", **csfont)
plt.ylabel("Generalization Loss", **csfont)
plt.legend()

#### Overlay Final Samples

In [ ]:
normal_final_samples = normal_data
tensor_final_samples = tensor_data

plt.figure(3, figsize=(10, 8))
fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(20, 5))
plt.suptitle("Planar-Normal: Post-Training", **csfont)

ax[0].scatter(normal_final_samples['post_training_samples'][:, -4], \
            normal_final_samples['post_training_samples'][:, -3], s=1., color='blue', label='normal');
ax[0].scatter(tensor_final_samples['post_training_samples'][:, -4], \
            tensor_final_samples['post_training_samples'][:, -3], s=1., color='red', label='tensor');
ax[0].grid(True)
ax[0].set_title("Dimension (8, 9)")

ax[1].scatter(normal_final_samples['post_training_samples'][:, -2], \
            normal_final_samples['post_training_samples'][:, -3], s=1., color='blue');
ax[1].scatter(tensor_final_samples['post_training_samples'][:, -2], \
            tensor_final_samples['post_training_samples'][:, -3], s=1., color='red');
ax[1].grid(True)
ax[1].set_title("Dimension (9, 10)")

ax[2].scatter(normal_final_samples['post_training_samples'][:, -2], \
            normal_final_samples['post_training_samples'][:, -1], s=1., color='blue', label='normal');
ax[2].scatter(tensor_final_samples['post_training_samples'][:, -2], \
            tensor_final_samples['post_training_samples'][:, -1], s=1., color='red', label='tensor');
ax[2].legend()
ax[2].grid(True)
ax[2].set_title("Last Dimension (10, 11)")
plt.grid(True)

#### Overlay Test Error

In [ ]:
plt.figure(5)

fig, ax = plt.subplots(1, 2, figsize=(15, 4))
plt.suptitle("Loss Profiles of Tensor vs. Normal");
ax[0].plot(grid, avg_test_tf, label="Tensor")
ax[0].plot(grid, avg_test_nf, label="Normal")
ax[0].set_title("Full Profile")
ax[0].set_ylabel("Test Loss")
ax[0].set_xlabel("Epoch")
ax[0].grid(True)

ax[1].plot(grid[50:], avg_test_tf[50:], label="Tensor")
ax[1].plot(grid[50:], avg_test_nf[50:], label="Normal")
ax[1].set_title("Zoomed In")
ax[1].set_ylabel("Test Loss")
ax[1].set_xlabel("Epoch")
ax[1].grid(True)
ax[1].legend()

# Aggregate Results (ResNet)

Plotting loss profiles over 10 seeds. The following sections must be run sequentially.

### GL 1d

In [ ]:
avg_test_normal = 0
avg_test_tensor = 0
for seed in np.arange(1, 5):
    tensor_file = "./report_aggregate/report/" + "seed{}_ginz1d_rk3_tf_resnet_report.mat".format(seed)
    normal_file = "./report_aggregate/report/" + "seed{}_ginz1d_nf_resnet_report.mat".format(seed)
    # load
    normal_data = scipy.io.loadmat(normal_file)
    tensor_data = scipy.io.loadmat(tensor_file)
    #plt.figure(1)
    #plt.plot(np.arange(len(data['training_loss'][0])), data['training_loss'][0])
    #plt.title("Training Error")
    plt.figure(2, figsize=(20, 8))
    normal_test_loss = np.append(normal_data['initial_loss'][0], normal_data['test_loss'][0])[0:121]
    tensor_test_loss = np.append(tensor_data['initial_loss'][0], tensor_data['test_loss'][0])[0:121]
    #test_loss = data['test_loss'][0]
    grid = np.arange(len(normal_test_loss))
    #error = np.random.normal(0, 2, size=test_loss.shape)
    plt.plot(grid, normal_test_loss, color='grey')
    plt.plot(grid, tensor_test_loss, color='black')
    #plt.fill_between(grid, test_loss-error, test_loss+error, color='grey')
    plt.grid(True)
    csfont = {'fontname':'Times New Roman'}
    plt.title("Rosenbrock Test Err.", **csfont)
    # averaged curve
    avg_test_tensor = avg_test_tensor + tensor_test_loss
    avg_test_normal = avg_test_normal + normal_test_loss
    
# plot averaged curve
#avg_train_nf = avg_train / 9
avg_test_nf = avg_test_normal / 4
avg_test_tf = avg_test_tensor / 4
plt.figure(2)
plt.plot(grid, avg_test_nf, label="( Normalizing Average )", color='yellow')
plt.plot(grid, avg_test_tf, label="( Tensorizing Average )", color='orange')
print("> Averaged Final Test Loss = {}".format(avg_test_nf[-1]))
print("> Averaged Final Test Loss = {}".format(avg_test_tf[-1]))
plt.xlabel("Epoch", **csfont)
plt.ylabel("Generalization Loss", **csfont)
plt.legend()

In [ ]:
normal_final_samples = normal_data
tensor_final_samples = tensor_data

plt.figure(3, figsize=(10, 8))
fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(20, 5))
plt.suptitle("ResNet-Normal: Post-Training", **csfont)

ax[0].scatter(normal_final_samples['post_training_samples'][:, -24], \
            normal_final_samples['post_training_samples'][:, -23], s=1., color='blue', label='normal');
ax[0].scatter(tensor_final_samples['post_training_samples'][:, -24], \
            tensor_final_samples['post_training_samples'][:, -23], s=1., color='red', label='tensor');
ax[0].grid(True)
ax[0].set_title("Dimension (23, 24)")

ax[1].scatter(normal_final_samples['post_training_samples'][:, -12], \
            normal_final_samples['post_training_samples'][:, -13], s=1., color='blue');
ax[1].scatter(tensor_final_samples['post_training_samples'][:, -12], \
            tensor_final_samples['post_training_samples'][:, -13], s=1., color='red');
ax[1].grid(True)
ax[1].set_title("Dimension (53, 54)")

ax[2].scatter(normal_final_samples['post_training_samples'][:, -2], \
            normal_final_samples['post_training_samples'][:, -1], s=1., color='blue', label='normal');
ax[2].scatter(tensor_final_samples['post_training_samples'][:, -2], \
            tensor_final_samples['post_training_samples'][:, -1], s=1., color='red', label='tensor');
ax[2].legend()
ax[2].grid(True)
ax[2].set_title("Last Dimension (63, 64)")
plt.grid(True)

In [ ]:
plt.figure(5)

fig, ax = plt.subplots(1, 2, figsize=(15, 4))
plt.suptitle("Ginzburg-Landau 1D: Loss Profiles of Tensor vs. Normal");
ax[0].plot(grid, avg_test_tf, label="Tensor")
ax[0].plot(grid, avg_test_nf, label="Normal")
ax[0].set_title("Full Profile")
ax[0].set_ylabel("Test Loss")
ax[0].set_xlabel("Epoch")
ax[0].grid(True)

ax[1].plot(grid[50:], avg_test_tf[50:], label="Tensor")
ax[1].plot(grid[50:], avg_test_nf[50:], label="Normal")
ax[1].set_title("Zoomed In")
ax[1].set_ylabel("Test Loss")
ax[1].set_xlabel("Epoch")
ax[1].grid(True)
ax[1].legend()

### GL 2d

#### Normal

In [ ]:
avg_test_normal = 0
avg_test_tensor = 0
for seed in np.arange(1, 10):
    normal_file = "./report_aggregate/report/" + "seed{}_ginz2d_nf_resnet_report.mat".format(seed)
    tensor_file = "./report_aggregate/report/" + "seed{}_ginz2d_rk3_tf_resnet_report.mat".format(seed)
    # load
    normal_data = scipy.io.loadmat(normal_file)
    tensor_data = scipy.io.loadmat(tensor_file)
    #plt.figure(1)
    #plt.plot(np.arange(len(data['training_loss'][0])), data['training_loss'][0])
    #plt.title("Training Error")
    plt.figure(2, figsize=(20, 8))
    normal_test_loss = np.append(normal_data['initial_loss'][0], normal_data['test_loss'][0])
    tensor_test_loss = np.append(tensor_data['initial_loss'][0], tensor_data['test_loss'][0])
    #test_loss = data['test_loss'][0]
    grid = np.arange(len(normal_test_loss))
    #error = np.random.normal(0, 2, size=test_loss.shape)
    plt.plot(grid, normal_test_loss, color='grey')
    plt.plot(grid, tensor_test_loss, color='black')
    #plt.fill_between(grid, test_loss-error, test_loss+error, color='grey')
    plt.grid(True)
    csfont = {'fontname':'Times New Roman'}
    plt.title("Rosenbrock Test Err.", **csfont)
    # averaged curve
    avg_test_tensor = avg_test_tensor + tensor_test_loss
    avg_test_normal = avg_test_normal + normal_test_loss
    
# plot averaged curve
#avg_train_nf = avg_train / 9
avg_test_nf = avg_test_normal / 9
avg_test_tf = avg_test_tensor / 9
plt.figure(2)
plt.plot(grid, avg_test_nf, label="( Normalizing Average )", color='yellow')
plt.plot(grid, avg_test_tf, label="( Tensorizing Average )", color='orange')
print("> Averaged Final Test Loss = {}".format(avg_test_nf[-1]))
print("> Averaged Final Test Loss = {}".format(avg_test_tf[-1]))
plt.xlabel("Epoch", **csfont)
plt.ylabel("Generalization Loss", **csfont)
plt.legend()

#### Overlay Final Samples

In [ ]:
normal_final_samples = normal_data
tensor_final_samples = tensor_data

plt.figure(3, figsize=(10, 8))
fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(20, 5))
plt.suptitle("ResNet-Normal: Post-Training", **csfont)

ax[0].scatter(normal_final_samples['post_training_samples'][:, -24], \
            normal_final_samples['post_training_samples'][:, -23], s=1., color='blue', label='normal');
ax[0].scatter(tensor_final_samples['post_training_samples'][:, -24], \
            tensor_final_samples['post_training_samples'][:, -23], s=1., color='red', label='tensor');
ax[0].grid(True)
ax[0].set_title("Dimension (23, 24)")

ax[1].scatter(normal_final_samples['post_training_samples'][:, -12], \
            normal_final_samples['post_training_samples'][:, -13], s=1., color='blue');
ax[1].scatter(tensor_final_samples['post_training_samples'][:, -12], \
            tensor_final_samples['post_training_samples'][:, -13], s=1., color='red');
ax[1].grid(True)
ax[1].set_title("Dimension (53, 54)")

ax[2].scatter(normal_final_samples['post_training_samples'][:, -2], \
            normal_final_samples['post_training_samples'][:, -1], s=1., color='blue', label='normal');
ax[2].scatter(tensor_final_samples['post_training_samples'][:, -2], \
            tensor_final_samples['post_training_samples'][:, -1], s=1., color='red', label='tensor');
ax[2].legend()
ax[2].grid(True)
ax[2].set_title("Last Dimension (63, 64)")
plt.grid(True)

#### Overlay Test Error

In [ ]:
plt.figure(5)

fig, ax = plt.subplots(1, 2, figsize=(15, 4))
plt.suptitle("Ginzburg-Landau 2D: Loss Profiles of Tensor vs. Normal");
ax[0].plot(grid, avg_test_tf, label="Tensor")
ax[0].plot(grid, avg_test_nf, label="Normal")
ax[0].set_title("Full Profile")
ax[0].set_ylabel("Test Loss")
ax[0].set_xlabel("Epoch")
ax[0].grid(True)

ax[1].plot(grid[50:], avg_test_tf[50:], label="Tensor")
ax[1].plot(grid[50:], avg_test_nf[50:], label="Normal")
ax[1].set_title("Zoomed In")
ax[1].set_ylabel("Test Loss")
ax[1].set_xlabel("Epoch")
ax[1].grid(True)
ax[1].legend()

In [ ]:
print(avg_test_tf[0], avg_test_tf[-1])
print(avg_test_nf[0], avg_test_nf[-1])

### Rosenbrock

#### Normal

In [ ]:
avg_test_normal = 0
avg_test_tensor = 0
for seed in np.arange(1, 10):
    normal_file = "./report_aggregate/report/" + "seed{}_rosen_nf_resnet_report.mat".format(seed)
    tensor_file = "./report_aggregate/report/" + "seed{}_rosen_rk5_tf_resnet_report.mat".format(seed)
    # load
    normal_data = scipy.io.loadmat(normal_file)
    tensor_data = scipy.io.loadmat(tensor_file)
    #plt.figure(1)
    #plt.plot(np.arange(len(data['training_loss'][0])), data['training_loss'][0])
    #plt.title("Training Error")
    plt.figure(2, figsize=(20, 8))
    normal_test_loss = np.append(normal_data['initial_loss'][0], normal_data['test_loss'][0])
    tensor_test_loss = np.append(tensor_data['initial_loss'][0], tensor_data['test_loss'][0])
    #test_loss = data['test_loss'][0]
    grid = np.arange(len(normal_test_loss))
    #error = np.random.normal(0, 2, size=test_loss.shape)
    plt.plot(grid, normal_test_loss, color='grey')
    plt.plot(grid, tensor_test_loss, color='black')
    #plt.fill_between(grid, test_loss-error, test_loss+error, color='grey')
    plt.grid(True)
    csfont = {'fontname':'Times New Roman'}
    plt.title("Rosenbrock Test Err.", **csfont)
    # averaged curve
    avg_test_tensor = avg_test_tensor + tensor_test_loss
    avg_test_normal = avg_test_normal + normal_test_loss
    
# plot averaged curve
#avg_train_nf = avg_train / 9
avg_test_nf = avg_test_normal / 9
avg_test_tf = avg_test_tensor / 9
plt.figure(2)
plt.plot(grid, avg_test_nf, label="( Normalizing Average )", color='yellow')
plt.plot(grid, avg_test_tf, label="( Tensorizing Average )", color='orange')
print("> Averaged Final Test Loss = {}".format(avg_test_nf[-1]))
print("> Averaged Final Test Loss = {}".format(avg_test_tf[-1]))
plt.xlabel("Epoch", **csfont)
plt.ylabel("Generalization Loss", **csfont)
plt.legend()

#### Overlay Samples

In [ ]:
normal_final_samples = normal_data
tensor_final_samples = tensor_data

plt.figure(3, figsize=(10, 8))
fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(20, 5))
plt.suptitle("ResNet-Normal: Post-Training", **csfont)

ax[0].scatter(normal_final_samples['post_training_samples'][:, -4], \
            normal_final_samples['post_training_samples'][:, -3], s=1., color='blue', label='normal');
ax[0].scatter(tensor_final_samples['post_training_samples'][:, -4], \
            tensor_final_samples['post_training_samples'][:, -3], s=1., color='red', label='tensor');
ax[0].grid(True)
ax[0].set_title("Dimension (8, 9)")

ax[1].scatter(normal_final_samples['post_training_samples'][:, -2], \
            normal_final_samples['post_training_samples'][:, -3], s=1., color='blue');
ax[1].scatter(tensor_final_samples['post_training_samples'][:, -2], \
            tensor_final_samples['post_training_samples'][:, -3], s=1., color='red');
ax[1].grid(True)
ax[1].set_title("Dimension (9, 10)")

ax[2].scatter(normal_final_samples['post_training_samples'][:, -2], \
            normal_final_samples['post_training_samples'][:, -1], s=1., color='blue', label='normal');
ax[2].scatter(tensor_final_samples['post_training_samples'][:, -2], \
            tensor_final_samples['post_training_samples'][:, -1], s=1., color='red', label='tensor');
ax[2].legend()
ax[2].grid(True)
ax[2].set_title("Last Dimension (10, 11)")
plt.grid(True)

#### Overlay Test Error

In [ ]:
plt.figure(5)

fig, ax = plt.subplots(1, 2, figsize=(15, 4))
plt.suptitle("Rosenbrock: Loss Profiles of Tensor vs. Normal");
ax[0].plot(grid, avg_test_tf, label="Tensor")
ax[0].plot(grid, avg_test_nf, label="Normal")
ax[0].set_title("Full Profile")
ax[0].set_ylabel("Test Loss")
ax[0].set_xlabel("Epoch")
ax[0].grid(True)

ax[1].plot(grid[:100], avg_test_tf[:100], label="Tensor")
ax[1].plot(grid[:100], avg_test_nf[:100], label="Normal")
ax[1].set_title("Zoomed In")
ax[1].set_ylabel("Test Loss")
ax[1].set_xlabel("Epoch")
ax[1].grid(True)
ax[1].legend()

In [ ]:
print(avg_test_tf[0], min(avg_test_tf))
print(avg_test_nf[0], min(avg_test_nf))

### Double Rosenbrock

#### Normal

In [ ]:
avg_test_normal = 0
avg_test_tensor = 0
for seed in np.arange(1, 10):
    normal_file = "./report_aggregate/report/" + "seed{}_double_rosen_nf_resnet_report.mat".format(seed)
    tensor_file = "./report_aggregate/report/" + "seed{}_double_rosen_rk4_tf_resnet_report.mat".format(seed)
    # load
    normal_data = scipy.io.loadmat(normal_file)
    tensor_data = scipy.io.loadmat(tensor_file)
    #plt.figure(1)
    #plt.plot(np.arange(len(data['training_loss'][0])), data['training_loss'][0])
    #plt.title("Training Error")
    plt.figure(2, figsize=(20, 8))
    normal_test_loss = np.append(normal_data['initial_loss'][0], normal_data['test_loss'][0])
    tensor_test_loss = np.append(tensor_data['initial_loss'][0], tensor_data['test_loss'][0])
    #test_loss = data['test_loss'][0]
    grid = np.arange(len(normal_test_loss))
    #error = np.random.normal(0, 2, size=test_loss.shape)
    plt.plot(grid, normal_test_loss, color='grey')
    plt.plot(grid, tensor_test_loss, color='black')
    #plt.fill_between(grid, test_loss-error, test_loss+error, color='grey')
    plt.grid(True)
    csfont = {'fontname':'Times New Roman'}
    plt.title("Rosenbrock Test Err.", **csfont)
    # averaged curve
    avg_test_tensor = avg_test_tensor + tensor_test_loss
    avg_test_normal = avg_test_normal + normal_test_loss
    
# plot averaged curve
#avg_train_nf = avg_train / 9
avg_test_nf = avg_test_normal / 9
avg_test_tf = avg_test_tensor / 9
plt.figure(2)
plt.plot(grid, avg_test_nf, label="( Normalizing Average )", color='yellow')
plt.plot(grid, avg_test_tf, label="( Tensorizing Average )", color='orange')
print("> Averaged Final Test Loss = {}".format(avg_test_nf[-1]))
print("> Averaged Final Test Loss = {}".format(avg_test_tf[-1]))
plt.xlabel("Epoch", **csfont)
plt.ylabel("Generalization Loss", **csfont)
plt.legend()

#### Plot Final Samples

In [ ]:
normal_final_samples = normal_data
tensor_final_samples = tensor_data

plt.figure(3, figsize=(10, 8))
fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(20, 5))
plt.suptitle("Planar-Normal: Post-Training", **csfont)

ax[0].scatter(normal_final_samples['post_training_samples'][:, -4], \
            normal_final_samples['post_training_samples'][:, -3], s=1., color='blue', label='normal');
ax[0].scatter(tensor_final_samples['post_training_samples'][:, -4], \
            tensor_final_samples['post_training_samples'][:, -3], s=1., color='red', label='tensor');
ax[0].grid(True)
ax[0].set_title("Dimension (8, 9)")

ax[1].scatter(normal_final_samples['post_training_samples'][:, -2], \
            normal_final_samples['post_training_samples'][:, -3], s=1., color='blue');
ax[1].scatter(tensor_final_samples['post_training_samples'][:, -2], \
            tensor_final_samples['post_training_samples'][:, -3], s=1., color='red');
ax[1].grid(True)
ax[1].set_title("Dimension (9, 10)")

ax[2].scatter(normal_final_samples['post_training_samples'][:, -2], \
            normal_final_samples['post_training_samples'][:, -1], s=1., color='blue', label='normal');
ax[2].scatter(tensor_final_samples['post_training_samples'][:, -2], \
            tensor_final_samples['post_training_samples'][:, -1], s=1., color='red', label='tensor');
ax[2].legend()
ax[2].grid(True)
ax[2].set_title("Last Dimension (10, 11)")
plt.grid(True)

#### Overlay Test Error

In [ ]:
plt.figure(5)

fig, ax = plt.subplots(1, 2, figsize=(15, 4))
plt.suptitle("Double Rosenbrock Loss Profiles of Tensor vs. Normal");
ax[0].plot(grid, avg_test_tf, label="Tensor")
ax[0].plot(grid, avg_test_nf, label="Normal")
ax[0].set_title("Full Profile")
ax[0].set_ylabel("Test Loss")
ax[0].set_xlabel("Epoch")
ax[0].grid(True)

ax[1].plot(grid[50:], avg_test_tf[50:], label="Tensor")
ax[1].plot(grid[50:], avg_test_nf[50:], label="Normal")
ax[1].set_title("Zoomed In")
ax[1].set_ylabel("Test Loss")
ax[1].set_xlabel("Epoch")
ax[1].grid(True)
ax[1].legend()

In [ ]:
print(avg_test_tf[0], min(avg_test_tf))
print(avg_test_nf[0], min(avg_test_nf))

In [ ]:
# delete all below
stats = scipy.io.loadmat("./utils/full_rank_stats.mat")
stats['gl1d']['std']

In [ ]:
data = scipy.io.loadmat("./data/gl1d_anti_full_rank.mat")

In [ ]:
data['X'].mean(1)

In [ ]:
data['X'].std(1)

In [ ]:
X = torch.Tensor(data['X']).T
likes = torch.log(torch.Tensor(data['likes']))

In [ ]:
np.log(data['likes'])

In [ ]:
def loss_func_init(x, prior_logpdf, targ_logpdf=utils.target.ginzburg_landau1d_anti_logpdf):
    """ evaluate initial KL divergence between posterior distribution (NF + prior) 
    and target. x is samples without flow. This is a Monte-Carlo estimation of the 
    log partition function. """
    return (prior_logpdf - targ_logpdf(x)).mean()


In [ ]:
pwd

In [ ]:
data = scipy.io.loadmat("../../gl1d_anti_full_rank.mat")

In [ ]:
X = data['X']
R = data['R'][0]
plt.plot(np.linspace(0, 1, 25), R*X[:,5], "-o", lw=2, color='orange'); plt.grid(True);
plt.plot(np.linspace(0, 1, 25), R*X[:,4], "-o", lw=2, color='blue'); plt.grid(True);

In [ ]:
# load tensorizing flow dataset
ginz_tf_dataset = utils.datasets.TensorizingFlowDataset("gl1d_samples_rk3.mat")
# initialize NF model
ginz_flow = NormalizingFlow(dim=ginz_tf_dataset.dim, blocks=RESNET_BLOCKS2_GL1D, \
                            flow_length=6)
# target PDF function
ginz1d_log_pdf = ginzburg_landau1d_logpdf

# begin training
ginz1d_report = train(ginz_tf_dataset, ginz_flow, ginz1d_log_pdf, 
          num_epochs=100,
          batch_size=256,
          verbose=True,
          lr=4e-4, 
          use_scheduler=True)

## Quick Testing

In [ ]:
data = scipy.io.loadmat("mixture_gaussian_nf_resnet_seed4.mat")['report']
data_tf = scipy.io.loadmat("mixture_gaussian_tf_resnet_seed4.mat")['report']
nf_test = data['test_loss'][0][0][0]
tf_test = data_tf['test_loss'][0][0][0]
print(nf_test[-1], tf_test[-1])

In [ ]:
plt.plot(nf_test, color='red')
plt.plot(tf_test, color='blue')

In [ ]:
nf_samples = data['post_training_samples'][0][0]
tf_samples = data_tf['post_training_samples'][0][0]


In [ ]:
plt.scatter(nf_samples[:, -2], nf_samples[:, -1], s=1.2, color='red')
plt.scatter(tf_samples[:, -2], tf_samples[:, -1], s=1.2, color='blue')
plt.xlim([-1, 1])
plt.ylim([-1, 1])